# Building a NanoGPT from Scratch: Attempting to Understand the Transformer Architecture
## 0. Introduction
My journey began in late 2024/early 2025 when I discovered Andrej Karpathy's video **Let's build GPT: from scratch, in code, spelled out**. The two-hour deep dive provided a clear and, low-level overview of the architecture. While I grasped the general framework of the Transformer, the inner workings of the Attention mechanism remained opaque.

The challenge wasn't just the architecture itself; it was my **foundation**. My memories of undergraduate linear algebra, & machine learning were nonexistent, as if by now those tokens were outside of my context window. necessitating a serious refresh on core concepts like **matrix multiplication, dot product, loss function, backpropagation, and tensor dimensionality**.

Throughout the year, I repeatedly returned to Karpathy’s explanation. **Each time, I peeled back a new layer of complexity**, steadily improving my grasp of the Transformer's operation. What started as a vague concept slowly transformed (hehe) into a detailed understanding.

This document is the culmination of that effort my attempt to synthesize everything I’ve learned about how these **"thinking machines"** work. I aimed for clarity and depth, and I hope it proves genuinely useful to others starting this same path.

This notebook provides a technical overview and guide for a minimal implementation of a Generative Pre-trained Transformer (**NanoGPT**), built using PyTorch. This project serves as a clear, educational reference for understanding the core components of modern large language models, from data preparation to the self-attention mechanism and text generation.

---

## 1. Data Acquisition and Character-Level Tokenization

The first step in any language modeling project is preparing the text data. For this notebook, we used a public domain work of literature: **Don Quijote de la Mancha**.

### Data Loading and Tokenization
The text was downloaded directly from Project Gutenberg using `curl`. A simple **character-level tokenizer** was implemented by mapping every unique character to an integer ID.

* The **vocabulary size** ($\boldsymbol{V}$ or `vocab_size`) is determined by the number of unique characters.
* The entire corpus is encoded into a long list of integers.

### Data Preparation and Dimensions
The encoded corpus is split into training and validation sets (90/10). For training, we sample batches of input sequences ($X$) and target sequences ($Y$).

Let $B$ be the batch size (which is implicitly set to `n_embd` in the `sample_data` call for the loss estimate) and $T$ be the context length (`block_size`).

* **Input Data:** The input batch $X$ is a tensor of shape $(B, T)$.
* **Target Data:** The target batch $Y$ is also a tensor of shape $(B, T)$, where $Y_{i, t}$ is the correct prediction for the token $X_{i, t+1}$.

### Sources

* **Attention Is All You Need**
* **LLM Visualization**
* **Understanding and Coding the Self-Attention Mechanism of Large Language Models From Scratch**
* **Andrej Karpathy's makemore/nanoGPT Video**
* **Language Models are Unsupervised Multitask Learners (GPT-2 Paper)**
* **Wikipedia: Byte-pair encoding**

In [ ]:
!curl -O https://www.gutenberg.org/files/2000/2000-0.txt

In [ ]:
%pip install numpy
%pip install torch

In [14]:
import torch
import numpy

# Check for CUDA availability and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU Name: NVIDIA GeForce RTX 3060


In [300]:
file_path = "./2000-0.txt"
with open(file_path, 'r', encoding='utf-8', newline='\n') as f:
    contents = f.read()

In [301]:
num_tokens = 500
offset = 3
print(contents[num_tokens*offset:num_tokens*offset+num_tokens])

nde se prosigue la narración de la desgracia de
nuestro caballero

Del donoso y grande escrutinio que el cura y el
barbero hicieron en la librería de nuestro ingenioso hidalgo

De la segunda salida de nuestro buen caballero don
Quijote de la Mancha

Del buen suceso que el valeroso don Quijote tuvo en
la espantable y jamás imaginada aventura de los molinos de viento, con
otros sucesos dignos de felice recordación

Donde se concluye y da fin a la estupenda batalla que
el gallardo viz


### Character-Level Tokenization and Vocab Creation

We first create a simple **character-level tokenizer**. This approach is simple and highly robust (never fails on an OOV word) but is computationally inefficient for large models due to sequence bloat.

1.  **Extract Vocabulary:** Find all unique characters in the corpus.
2.  **Mapping:** Create lookup tables (`stoi` for string-to-integer and `itos` for integer-to-string).

In [303]:
# create a mapping from characters to integers
chars = sorted(list(set(contents)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }

itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [304]:
a = "Sancho Panza "
print(encode(a))
print(decode(encode(a)))

[48, 58, 71, 60, 65, 72, 2, 45, 58, 71, 83, 58, 2]
Sancho Panza 


### Batching Function

The **Transformer** operates on fixed-length chunks. We define a function to sample random batches of input ($X$) and target ($Y$) sequences.

* **Context Length ($\boldsymbol{T}$):** `block_size` (e.g., 32) is the maximum sequence length the model processes at once.
* **Batch Size ($\boldsymbol{B}$):** The number of independent sequences processed in parallel.

The target $\boldsymbol{Y}$ is simply the input $\boldsymbol{X}$ shifted one position to the right, as the model predicts the next token in the sequence.

In [6]:
def split_data(training_data: str, split_ratio: float = 0.9) -> tuple[str,str]:
    training_data_length = len(training_data)
    marker = int(split_ratio * training_data_length)
    return training_data[:marker], training_data[marker:]

In [7]:
import torch
def sample_data(data, len_samples:int=16, num_samples:int=8) -> torch.Tensor:
    len_data = len(data)
    random_indices = torch.randint(0, len_data - len_samples, (num_samples,))
    sample_x = torch.stack([
        torch.tensor(data[r_ind:r_ind+len_samples]) 
        for r_ind in random_indices
    ])
    sample_y = torch.stack([
        torch.tensor(data[r_ind+1:r_ind+1+len_samples]) 
        for r_ind in random_indices
    ])
    return sample_x, sample_y

In [8]:
encoded_contents = encode(contents)
training_data, validation_data = split_data(encoded_contents)

## 2. The Transformer Block and Attention Mechanism

The core of the NanoGPT is the **Transformer Block**, based on the seminal "Attention Is All You Need" paper. This is a decoder-only architecture, characteristic of GPT models.

### The NanoGPT Architecture
The model uses several learned components to transform the input:
1.  **Token Embeddings:** Maps the input IDs $(B, T)$ to a continuous space of size $C$ (`embedding_dim = 2048`).
    $$\text{Token Embeddings}: (B, T) \rightarrow (B, T, C)$$
2.  **Positional Embeddings:** A learned encoding added to the token embeddings to provide sequential context, as attention itself is order-agnostic.
3.  **Final Input Vector:** The sum of the token and positional embeddings forms the final input vector to the blocks.

### Causal Self-Attention and Matrix Sizes
The `AttentionHead` implements the scaled dot-product attention, modified for causality (GPT-style).

Let $H$ be the size of a single attention head (`head_size`). $H = C / N_h$, where $N_h$ is the number of heads (`num_heads = 12`).

1.  **Linear Projections (Q, K, V):** The input vector $x$ (shape $(B, T, C)$) is projected into Query ($\boldsymbol{Q}$), Key ($\boldsymbol{K}$), and Value ($\boldsymbol{V}$) matrices.
    $$\boldsymbol{Q}, \boldsymbol{K}, \boldsymbol{V} = W_Q(x), W_K(x), W_V(x) \rightarrow (B, T, H)$$
2.  **Attention Logits ($\Omega$):** The raw attention scores ($\Omega$) are calculated by the dot product of $Q$ and $K^T$. This score determines how much token $i$ should pay attention to token $j$.
    $$\Omega = \boldsymbol{Q}\boldsymbol{K}^T \rightarrow (B, T, H) \times (B, H, T) \rightarrow (B, T, T)$$
3.  **Scaling and Masking:** $\Omega$ is scaled by $\frac{1}{\sqrt{H}}$ to stabilize gradients. A lower-triangular mask (`tril`) is applied, setting entries corresponding to future tokens to $-\infty$. This is the **causal mask**, enforcing that the prediction for token $t$ can only depend on tokens $1$ through $t$.
4.  **Attention Weights ($W$):** The masked logits are passed through a softmax function, resulting in the attention weights $W$. These weights are the probability distribution over past tokens for the current token.
    $$W = \text{softmax}(\Omega_{\text{masked}} / \sqrt{H}) \rightarrow (B, T, T)$$
5.  **Context Vector (Output):** The weights $W$ are multiplied by the Value matrix $V$ to create the context vector, which is the output of the attention head.
    $$\text{Output} = W\boldsymbol{V} \rightarrow (B, T, T) \times (B, T, H) \rightarrow (B, T, H)$$

The **Multi-Head Attention** module runs $N_h$ heads in parallel, concatenates their outputs (shape $(B, T, N_h \cdot H)$), and projects the result back to $C$ using a final linear layer.

The **TransformerBlock** wraps this, applying Layer Normalization (Post-LN style), as originally implemented in **Attention is all you need** and **residual connections**: $x' = x + \text{LayerNorm}(\text{MultiHead}(x))$. This structure ensures gradients can flow easily through the network.

---

In [112]:
import torch.nn as nn
class AttentionHead(nn.Module):
    def __init__(self, n_embd:int, head_size:int, block_size:int, dropout:int=0):
        super().__init__()
        self.head_size = head_size
        self.W_query = nn.Linear(n_embd, head_size, bias=False)
        self.W_key = nn.Linear(n_embd, head_size, bias=False)
        self.W_value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B,T,C = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        omega = queries @ keys.mT # * C**-0.5

        omega_masked = omega.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

        attention_weights = nn.functional.softmax(omega_masked / self.head_size**0.5, dim=-1)
        wei = self.dropout(attention_weights)

        context_vector = wei @ values # (num_samples, num_words, num_embeddings_per_word)

        return context_vector

class MultiAttentionHead(nn.Module):
    def __init__(self, num_heads:int, n_embd:int, head_size:int, block_size:int, dropout:int=0.0):
        super().__init__()
        self.attention_blocks = nn.ModuleList([AttentionHead(n_embd, head_size, block_size) for _ in range(num_heads)])
        self.linear_projection = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        attention_weights = torch.cat([head(x) for head in self.attention_blocks], dim=-1)
        lin = self.linear_projection(attention_weights)
        
        return self.dropout(lin)

class FeedForwardNetwork(nn.Module):
    def __init__(self, n_embd:int, block_size:int, dropout:int=0.0):
        super().__init__()
        self.ln1 = nn.Linear(n_embd, 4 * n_embd)
        self.relu = nn.ReLU()
        self.ln2 = nn.Linear(4 * n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.ln1(x)
        x = self.relu(x)
        x = self.ln2(x)
        return self.dropout(x)

class TransformerBlock(nn.Module):
    def __init__(self, num_heads:int, n_embd:int, head_size:int, block_size:int):
        super().__init__()
        self.multi_head_attention = MultiAttentionHead(num_heads, n_embd, head_size, block_size)
        self.ffn = FeedForwardNetwork(n_embd, block_size)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #x = x + self.multi_head_attention(self.ln1(x))
        #x = x + self.ffn(self.ln2(x))

        x = x + self.ln1(self.multi_head_attention(x))
        x = x + self.ln2(self.ffn(x))
        return x


In [113]:
# A very simple and stupid GPT model!
class NanoGPT(nn.Module):
    def __init__(self, num_layers:int, vocab_size:int, embedding_dim:int, num_heads:int, block_size:int):
        super().__init__()
        head_size = embedding_dim // num_heads

        self.block_size = block_size
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.positional_embeddings = nn.Embedding(block_size,embedding_dim)

        self.blocks = nn.Sequential(*[TransformerBlock(num_heads, embedding_dim, head_size, block_size) for _ in range(num_layers)])
    
        self.ln1 = nn.LayerNorm(embedding_dim)
        self.lm_head = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, input_ids: torch.Tensor, targets: torch.Tensor|None=None) -> torch.Tensor:
        B,T = input_ids.shape
        word_embeddings = self.embedding(input_ids)
        pos = torch.arange(T)
        pos = pos.to(device)
        
        positional_embeddings = self.positional_embeddings(pos)

        final_vector = word_embeddings + positional_embeddings

        x = self.blocks(final_vector) # (B,T,C)
        x = self.ln1(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        num_samples, num_words, num_embeddings = logits.shape
        
        if targets == None:
            loss = None
        else:
            logits = logits.view(num_samples * num_words, num_embeddings)
            targets = targets.view(num_samples * num_words)
            loss = nn.functional.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = nn.functional.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [11]:
@torch.no_grad()
def estimate_loss(model, train_data, val_data, block_size, n_embd, eval_iters: int = 20):
    # Dictionaries to store the average loss for each split
    out = {}
    model.eval() # Set model to evaluation mode (crucial for dropout/BatchNorm)

    for split_name, data in [('train', train_data), ('val', val_data)]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            # Sample data from the current split (train or val)
            xb, yb = sample_data(data, block_size, n_embd)
            
            xb = xb.to(device)
            yb = yb.to(device)
            # Forward pass to get logits and loss
            logits, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split_name] = losses.mean()

    model.train() # Set model back to training mode
    return out

## 3. Training Optimization: Loss and Gradient Descent

### Loss Function
The model's final layer, the language model head (`lm_head`), produces **logits** of shape $(B, T, V)$. These logits represent the raw, unnormalized scores for every possible next character in the vocabulary at every time step.

The model uses the standard **Cross-Entropy Loss** for classification tasks, comparing the predicted logits against the true target tokens $Y$.

$$\text{Loss} = \text{CrossEntropyLoss}(\text{Logits}, \text{Targets})$$

To calculate this, the logits and targets are often reshaped into two-dimensional vectors, effectively treating all $B \times T$ predictions in the batch as independent classification problems:
* $\text{Logits}: (B \cdot T, V)$
* $\text{Targets}: (B \cdot T)$

### Gradient Descent and Optimization
The goal of training is to find the model parameters (weights) that minimize this loss. This is achieved using **Gradient Descent**, specifically the **AdamW optimizer**.

1.  **Forward Pass:** Calculate the loss.
2.  **Backward Pass (Backpropagation):** The `loss.backward()` call computes the gradient (derivative) of the loss with respect to every single parameter in the model.
3.  **Parameter Update:** The `optimizer.step()` function uses these calculated gradients to update the model's parameters in the direction that decreases the loss, aiming for convergence.

---

In [12]:
num_layers = 4
n_embd = 248
num_heads = 12
block_size = 2048 >> 2
gpt = NanoGPT(num_layers, vocab_size, n_embd, num_heads, block_size)
gpt.to(device)
print(sum(p.numel() for p in gpt.parameters())/1e6, 'M parameters')

3.110522 M parameters


In [13]:
optimizer = torch.optim.AdamW(gpt.parameters(), lr=1e-3)
total_steps = 1000  
for steps in range(total_steps): # increase number of steps for good results...
    # sample a batch of data

    xb, yb = sample_data(training_data, block_size, n_embd)
    xb = xb.to(device)
    yb = yb.to(device)
    # evaluate the loss
    logits, loss = gpt(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if steps % (total_steps//10) == 0 or steps ==  total_steps - 1:
        print(loss.item())
    del xb, yb, logits, loss

4.827203750610352
2.2913737297058105
2.2512147426605225
2.1931519508361816
2.0742852687835693
1.6983325481414795
1.5868492126464844
1.4134609699249268
1.313541054725647
1.2511333227157593
1.2177703380584717


In [15]:
starting_idx = torch.zeros((1, 1), dtype=torch.long, device=device)
print(starting_idx)
starting_idx = torch.tensor([encode("Quien eres?")], dtype=torch.long, device=device)
print(starting_idx)
print(decode(gpt.generate(idx = starting_idx, max_new_tokens=1000)[0].tolist()))

tensor([[0]], device='cuda:0')
tensor([[46, 78, 66, 62, 71,  2, 62, 75, 62, 76, 28]], device='cuda:0')
Quien eres?
Ella despa está ser riqueza la gente que se mucho en venturas,
así, dejando que ya a gobernador, y se aproprese complasen sangre si
estavores todos los ojores, y su alcaún necesica sobre vuestra merced
dádis por quien no lo viniese yo que debe en de todas vestidas suerte;
los casaden más se salamente, y es especiallos no regaló a vino alguna que le
habían truinos como jurarona, y en cuanto de peligro premoza el de Caballero—
sirve lo que se me haga malicia; mas, no le pude todo o a cabo a don
llorcabar, como si agude); la cual avella y que precios ágoles, tanto esto
particulo para conocerle había, que yo la bogará en la mance, no por deseá
sentimiento mil de Cardenio, mas cuitan, y que sean anastase,
túnomplejo,
que yo puesto es cosas fortales de Grióneril

Dulcincia. Todavio te tomamos in paso Sancho ¡dicieron esas
Basilio por persona y justición también los hizo
de fin l

## 4. Why Character-Level Tokenization is Inefficient

The character-level approach, while simple and robust against out-of-vocabulary (OOV) words, is highly **inefficient** for training and running modern Transformer-based LLMs:

1.  **Excessive Sequence Length (Sequence Bloat):** A single common word requires multiple tokens (characters). This maximizes the sequence length ($T$) for any given text. Texts encoded at the character level are often **3 to 5 times longer** than those encoded using subword methods.
2.  **Quadratic Computational Cost:** The self-attention mechanism, the core operation of the Transformer, has a computational complexity that is **quadratic** with respect to the sequence length, $O(T^2)$. If the sequence length is four times longer, the computation for attention is $\sim 16$ times slower.
3.  **Reduced Effective Context Window:** Since LLMs have a fixed maximum context length, excessive tokenization limits the number of actual words the model can "see" at once, hindering its ability to capture **long-range dependencies** necessary for high-quality language understanding and generation.
4.  **Low Information Density:** Individual character tokens carry very **low semantic meaning**. The model must dedicate substantial capacity to learning the basic compositional structure of words rather than focusing on high-level linguistic relationships.

---

## 5. Next Steps: Implementing Byte Pair Encoding (BPE)

To overcome the inefficiencies of character-level tokenization, an implementation of the **Byte Pair Encoding (BPE)** algorithm was written as the next step. BPE, as used in models like GPT-2, balances character-level flexibility with word-level efficiency.

The BPE implementation in the notebook followed the general algorithm described on **Wikipedia**:
1.  **Iterative Merge:** Start with the input string represented by bytes.
2.  **Substitution:** Recursively identify the most frequent adjacent pair of tokens and replace them with a new, single token ID (starting from value 256).
3.  **Dictionary:** The substitution is recorded in a `dictionary` mapping the new token ID to the two tokens it replaces, allowing for decoding.

### Note on GPT-2 Paper and BPE
The implemented BPE was a pure algorithmic approach and **did not** incorporate the specific byte-level exception mentioned in the GPT-2 paper. This exception is typically added to prevent the merging of common words with preceding or trailing punctuation (e.g., separating `dog` from `.`, `!`, or `?`) to preserve valuable vocabulary slots and improve compression efficiency.

---

## 5.1 Byte Pair Encoding Training

The **BPE Encoder** section of the notebook implements a **subword tokenization** algorithm. This is a crucial step in language models like GPT, as it balances the need for a small, manageable vocabulary size with the ability to represent rare or unseen words (the "out-of-vocabulary" or OOV problem). 

### The Role of BPE

* **Balancing Vocabulary and Sequence Length:**
    * Using every single word as a token results in a huge, sparse vocabulary, and many words are rarely seen.
    * Using individual characters results in a tiny vocabulary, but the input **sequence length** becomes very long, increasing the computational cost of the Transformer's attention mechanism.
* **The BPE Solution:** BPE tokenizes text into **subwords** (e.g., `_pre` + `vent` + `ing`), where the vocabulary consists of common characters, character pairs, and subwords. This keeps the final vocabulary size reasonable while keeping the average input sequence length shorter than character-level tokenization.

### The "Training" Process

The BPE training process is an iterative, greedy algorithm that learns the most efficient way to combine base units (initial characters) into larger, frequent tokens.

1.  **Initialization:** The process begins with a base vocabulary containing every unique **character** found in the entire training text.
2.  **Frequency Analysis:** The training loop repeatedly scans the entire corpus to find the **most frequent adjacent pair** of tokens (e.g., `('t', 'h')`).
3.  **Merge Rule Creation:** Once the most frequent pair is identified, a **merge rule** is created, and the two tokens are replaced everywhere in the corpus with a new, single token (e.g., `('t', 'h')` is merged into `'th'`). This new token is added to the vocabulary.
4.  **Iteration:** This merging process is repeated for a specified number of merge operations (e.g., 50,000 steps) or until a target vocabulary size is reached.
5.  **The `merge_dict`:** The final ordered set of merge rules (the `merge_dict` you see in the notebook) is the entire learned **BPE encoder**. This dictionary is what is used by the encoding and decoding functions to process new, unseen text.

### Examples of LLM Vocabulary Sizes

The final size of the BPE vocabulary is determined by the number of merge operations performed on the base character set, striking a balance between compression efficiency and vocabulary size (the size of the embedding matrix).

| Model Family | Vocabulary Size | Tokenizer Base |
| :--- | :--- | :--- |
| **GPT-2** | **50,257** | Byte-level BPE |
| **GPT-3 / GPT-3.5** | **~50,000 - 100,256** | Byte-level BPE (OpenAI has since standardized on a larger, more efficient vocabulary, often **100,256**, for modern models like GPT-4, referred to as `cl100k_base`). |
| **Llama 1 / Llama 2** | **32,000** | SentencePiece (BPE variant) |
| **Llama 3** | **128,000** | Tiktoken-based BPE (a significant increase for multilingual support) |

---

In [138]:
def bp_encoding(input_bytes:list[bytes], dictionary={}, unused=256, level=0, max_level=20, vocab_size=50000) -> list[int]:
    while level < max_level or unused != vocab_size:
        len_bytes = len(input_bytes)
        byte_pair_bin = {}
    
        # here walk through the input and calculate the byte pairs
        for i in range(len_bytes-1):
            packed_bp = input_bytes[i] | (input_bytes[i+1] << 32)
    
            if packed_bp not in byte_pair_bin:
                byte_pair_bin[packed_bp] = 1
            else:
                byte_pair_bin[packed_bp] += 1
    
        # here we walk through the input in order to replace the byte pair
        index = 0
        top_bp = max(byte_pair_bin, key=byte_pair_bin.get)
        v_size = max(byte_pair_bin)
    
        if byte_pair_bin[top_bp] == 1 :
            break
        
        encoded = []
        update_unused = False
        while index < len_bytes:
            if index == len_bytes - 1:
                encoded.append(input_bytes[index])
                break
                
            # why pack the bp? why not use a tuple? because we want to go fast
            packed_bp = input_bytes[index] | (input_bytes[index+1] << 32)
    
            if packed_bp == top_bp:
                encoded.append(unused)
                dictionary[unused] = top_bp
                index += 2
                update_unused = True
            else:
                encoded.append(input_bytes[index])
                index += 1
        
        if update_unused:
            unused += 1

        level += 1
        input_bytes = encoded
    
    return input_bytes

def bp_encode_train(input_str:str, max_level=20) -> [str, dict[int,int]]:
    dictionary={}
    input_bytes = [int(b) for b in input_str.encode("utf-8")]

    encoded = bp_encoding(input_bytes, dictionary, max_level=max_level)
    return encoded, dictionary

def bp_encode(input_str:str, dictionary:dict[int,int]) -> list[int]:
    symbols_to_compress = dictionary.keys()
    input_bytes = list(input_str.encode("utf-8"))
    
    for byte_to_compress in symbols_to_compress:
        index = 0
        new_tokens = []
        
        while index < len(input_bytes):
            if index == len(input_bytes) - 1:
                new_tokens.append(input_bytes[index])
                break
            
            b0 = input_bytes[index]
            b1 = input_bytes[index + 1]
    
            packed_bp = b0 | (b1 << 32)
    
            if packed_bp == byte_to_compress:
                new_tokens.append(dictionary[packed_bp])
                index += 2
            else:
                new_tokens.append(b0)
                index += 1
        
        input_bytes = new_tokens

    return input_bytes  

In [85]:
def bp_decode(input_tokens: list[int], dictionary: dict) -> list[int]:
    decoded_tokens = list(input_tokens)

    # Sort the dictionary keys (new symbols) in descending order.
    # We must reverse the substitutions, so we start with the last encoded symbol.
    symbols_to_expand = sorted(dictionary.keys(), reverse=True)

    for symbol in symbols_to_expand:
        # Get the packed pair integer for the current symbol
        packed_pair = dictionary[symbol]
        
        # Token_2 (MSB): Shift right by 32
        # Token_1 (LSB): Use bitwise AND with 0xFFFF to isolate the lower 32 bits
        token_2 = packed_pair >> 32
        token_1 = packed_pair & 0xFFFFFFFF
        original_pair = [token_1, token_2] # Note: Order is LSB then MSB

        new_decoded_tokens = []
        i = 0
        while i < len(decoded_tokens):
            if decoded_tokens[i] == symbol:
                # Replace the encoded symbol with its original two tokens
                new_decoded_tokens.extend(original_pair)
            else:
                new_decoded_tokens.append(decoded_tokens[i])
            i += 1
        
        decoded_tokens = new_decoded_tokens

    return bytes(decoded_tokens).decode("utf-8")

In [ ]:
input_str = "😀😃😄😁😆😅🤣😂🙂🙃"
print(input_str)
e, dictionary = bp_encode_train(contents, max_level=25)
rev_dictionary = {v:k for k,v in dictionary.items()}
e = bp_encode(input_str, rev_dictionary)
print(e)
print(bp_decode(e, dictionary))

## Numba: Just-In-Time (JIT) Compilation Motivation

The library **Numba** is used to accelerate the training, encoding, and decoding portions of the Byte Pair Encoding Python code.

### What is Numba?

**Numba** is an open-source **Just-In-Time (JIT) compiler** that translates a subset of Python and NumPy code into fast, native **machine code** at runtime using the **LLVM** compiler infrastructure.

When a function is decorated with `@numba.njit` (an alias for `@jit(nopython=True)`), Numba:
1.  **Infers Types:** Analyzes the Python function and determines the specific data types of all variables.
2.  **Compiles:** Translates the Python bytecode directly into optimized machine code, specifically tailored to your CPU.
3.  **Executes at Native Speed:** Executes the compiled machine code, which runs **entirely without the Python interpreter's overhead** (in "nopython mode").

### Why Use Numba?

Numba is incredibly useful for speeding up Python code that is typically slow:

* **Eliminates Interpreter Overhead:** It bypasses the overhead of the CPython interpreter, which is the main bottleneck for numerical code and explicit loops in standard Python.
* **Accelerates Loops and NumPy:** It provides significant speedups for numerical code that heavily relies on `for` loops and NumPy array operations, often achieving **orders of magnitude (10x-100x) speedup** compared to pure Python.
* **Parallelism:** It can automatically parallelize certain loops to leverage multiple CPU cores with minimal code changes.

### The Trade-off: Code vs. Speed

Using Numba involves an engineering **trade-off** between developer effort and execution speed:

| Language/Method | Developer Effort | Performance |
| :--- | :--- | :--- |
| **Pure Python** | Low (maximum flexibility) | Slow (due to interpreter overhead) |
| **Numba JIT** | **Low-to-Medium** (add decorator, restrict to Numba-supported Python subset) | **Very Fast** (approaches compiled speeds) |
| **Pure C/C++** | Very High (manual memory, C-API, complex compilation) | Fastest (the theoretical ceiling) |

The motivation for using Numba is that it provides a **massive performance gain** with **minimal coding overhead**:

In [90]:
import numpy as np
from numba import njit, types
from numba.typed import Dict, List

@njit
def bp_encoding_train_numba(
    input_array: List, 
    dictionary: Dict = None, 
    unused=256, 
    level=0, 
    max_level=20, 
    vocab_size=50000
) -> List:
    
    # Initialize dictionary if not provided (needed for type stability)
    if dictionary is None:
        dictionary = Dict.empty(
            key_type=types.int64,
            value_type=types.int64
        )
    
    current_tokens = input_array

    while level < max_level and unused < vocab_size:
        len_tokens = len(current_tokens)
        
        # 1. Calculate the byte pairs (packed_bp)
        byte_pair_bin = Dict.empty(
            key_type=types.int64, 
            value_type=types.int64
        )
        
        for i in range(len_tokens - 1):
            packed_bp = current_tokens[i] | (current_tokens[i+1] << 32)
            
            # Numba dictionary operations
            if packed_bp not in byte_pair_bin:
                byte_pair_bin[packed_bp] = 1
            else:
                byte_pair_bin[packed_bp] += 1

        
        # Check for empty tokens or only single pairs
        if not byte_pair_bin:
            break

        # 2. Find the most frequent byte pair (top_bp)
        # Numba does not support max(dict, key=dict.get). We must iterate manually.
        max_count = 0
        top_bp = -1 
        for bp, count in byte_pair_bin.items():
            if count > max_count:
                max_count = count
                top_bp = bp

        
        # If the most frequent pair only appears once, we stop merging.
        if max_count == 1:
            break

        # 3. Replace the top byte pair
        encoded_list = List.empty_list(types.int64)
        index = 0
        update_unused = False
        
        while index < len_tokens:
            if index == len_tokens - 1:
                encoded_list.append(current_tokens[index])
                break
            
            # Pack the current pair, should be a pretty fast way of doing this
            packed_bp = current_tokens[index] | (current_tokens[index+1] << 32)
            
            if packed_bp == top_bp:
                encoded_list.append(unused)
                dictionary[unused] = top_bp
                index += 2
                update_unused = True
            else:
                encoded_list.append(current_tokens[index])
                index += 1
        
        # Update state for the next iteration
        if update_unused:
            unused += 1

        level += 1
        
        current_tokens = encoded_list
    
    return current_tokens

In [91]:
def bp_encoding_wrapper(input_str: str, max_level=20, vocab_size=50000) -> tuple[List, Dict]:
    # 1. Convert the input string to a sequence of byte-level integers (0-255).
    byte_sequence = input_str.encode('utf-8')
    
    input_array_np = np.frombuffer(byte_sequence, dtype=np.uint8).astype(np.int64)
    input_array = List(input_array_np)

    # 2. Initialize the Numba Typed Dictionary for storing merges.
    dictionary = Dict.empty(
        key_type=types.int64,
        value_type=types.int64
    )
    
    # 3. Call the njit-compiled function
    encoded_tokens = bp_encoding_train_numba(
        input_array, 
        dictionary=dictionary, 
        unused=256, 
        level=0, 
        max_level=max_level, 
        vocab_size=vocab_size
    )
    
    return encoded_tokens, dictionary

In [92]:
def bp_encode(input_str:str, dictionary:Dict) -> np.ndarray:
    byte_sequence = input_str.encode('utf-8')
    input_array_np = np.frombuffer(byte_sequence, dtype=np.uint8).astype(np.int64)
    input_array = List(input_array_np)
    return bp_encode_numba(input_array, dictionary)

@njit
def bp_encode_numba(input_array:List, dictionary:Dict) -> List:
    symbols_to_compress = dictionary.keys()

    for byte_to_compress in symbols_to_compress:
        index = 0
        new_tokens = List.empty_list(types.int64)
        
        while index < len(input_array):
            if index == len(input_array) - 1:
                new_tokens.append(input_array[index])
                break
            
            b0 = input_array[index]
            b1 = input_array[index + 1]
    
            packed_bp = b0 | (b1 << 32)
    
            if packed_bp == byte_to_compress:
                new_tokens.append(dictionary[packed_bp])
                index += 2
            else:
                new_tokens.append(b0)
                index += 1
        
        input_array = new_tokens

    return input_array  

In [93]:
@njit
def reverse_numba_dict(dictionary: Dict) -> Dict:
    """
    Creates a new Numba typed Dict with keys and values swapped.
    
    Args:
        dictionary: The original Numba Dict (Token ID -> Packed Pair).
        
    Returns:
        A new Numba Dict (Packed Pair -> Token ID).
    """
    # 1. Create a new empty Dict. The key/value types are swapped compared to the original.
    # Original: key=int64 (Token ID), value=int64 (Packed Pair)
    # Reverse:  key=int64 (Packed Pair), value=int64 (Token ID)
    rev_dictionary = Dict.empty(
        key_type=types.int64,
        value_type=types.int64
    )
    
    # 2. Iterate manually and assign the swapped key-value pairs
    for token_id, packed_pair in dictionary.items():
        # packed_pair becomes the key, token_id becomes the value
        rev_dictionary[packed_pair] = token_id
        
    return rev_dictionary

In [155]:
@njit
def bp_decode_numba(input_tokens: List, dictionary: Dict) -> List:
    decoded_tokens = input_tokens

    # Sort the dictionary keys (new symbols) in descending order.
    # We must reverse the substitutions, so we start with the last encoded symbol.
    symbols_to_expand = sorted(dictionary.keys(), reverse=True)

    for symbol in symbols_to_expand:
        # Get the packed pair integer for the current symbol
        packed_pair = dictionary[symbol]
        
        # Token_2 (MSB): Shift right by 32
        # Token_1 (LSB): Use bitwise AND with 0xFFFF to isolate the lower 32 bits
        token_2 = packed_pair >> 32
        token_1 = packed_pair & 0xFFFFFFFF
        original_pair = List.empty_list(types.int64)
        original_pair.append(token_1) 
        original_pair.append(token_2) # Note: Order is LSB then MSB

        new_decoded_tokens = List.empty_list(types.int64)
        
        i = 0
        while i < len(decoded_tokens):
            if decoded_tokens[i] == symbol:
                # Replace the encoded symbol with its original two tokens
                new_decoded_tokens.extend(original_pair)
            else:
                new_decoded_tokens.append(decoded_tokens[i])
            i += 1
        
        decoded_tokens = new_decoded_tokens

    return decoded_tokens

def bp_decode(input_tokens: List, dictionary: Dict):
    return bytes(bp_decode_numba(input_tokens, dictionary)).decode("utf-8")

## Byte Pair Encoding (BPE) Training Summary

The Byte Pair Encoding (BPE) algorithm was trained on the **Don Quijote de la Mancha** corpus to create an efficient subword tokenizer. The goal was to balance a manageable vocabulary size with better text compression than character-level tokenization.

* **Total Merge Iterations:** The BPE training was executed for a maximum of **10,000 merge operations** (`max_level=10000`).
* **Final Vocabulary Size:** The training process resulted in a total learned vocabulary size of **10,255** tokens (determined by the base 255 byte tokens plus the 10,000 merges).
    * This is confirmed by the output: `vocab_size: 10255`.

In [116]:
final_tokens, merge_dict = bp_encoding_wrapper(contents, max_level=10000)
merge_dict_rev = reverse_numba_dict(merge_dict)
print(f"vocab_size: {max(merge_dict)}")

10255


In [122]:
import pickle
file_path = "./bpe_encoding_vocab.pkl"
try:
    with open(file_path, 'wb') as file:
        # Save the dictionary containing all data to the file
        pickle.dump(dict(merge_dict), file)
    print(f"✅ Object successfully serialized and saved to '{file_path}'")
except Exception as e:
    print(f"An error occurred during serialization: {e}")

✅ Object successfully serialized and saved to './bpe_encoding_vocab.pkl'


In [224]:
import colorsys

def generate_ansi_colors(N: int) -> list[str]:
    """
    Generates N distinct ANSI 256-color escape codes for terminal foreground coloring.
    """
    
    # Total number of colors available in the 6x6x6 color cube
    MAX_COLORS = 216
    
    if N > MAX_COLORS:
        print(f"Warning: Cannot generate {N} distinct colors from the 216-color cube.")
        N = MAX_COLORS
        
    START_CODE = 16
    
    # Calculate the step size to space the N colors evenly across the available range
    step = MAX_COLORS // N if N > 0 else 1
    
    ansi_codes = []
    
    for i in range(N):
        # Calculate the color index (16 to 231)
        color_index = START_CODE + (i * step)
        
        # Ensure the index doesn't exceed the color cube range (231)
        if color_index > 231:
            color_index = START_CODE + (color_index % (231 - START_CODE))
        
        ansi_code = f'\033[48;5;{color_index}m'
        ansi_codes.append(ansi_code)
        
    return ansi_codes

NUM_COLORS = 50
RESET = '\033[0m'


## Tokenizer Visualization and Language Efficiency

The visualization cell demonstrates the fundamental efficiency of a tokenizer trained on a specific language (in this case, **Spanish**) when processing text in that language, versus a new language (English).

### Why Spanish Compresses Better

The BPE training is a **greedy, data-driven process**. It learns to combine the most frequent adjacent byte/token pairs in the training corpus into new, single subword tokens.

1.  **High-Frequency Merges (Spanish):** The training text is entirely in Spanish. Consequently, the tokenizer learns common Spanish pairs and subwords like `por`, `que`, `el`, `cielo`, and `azul?`.
    * When the Spanish sentence **"por que es el cielo azul?"** is tokenized, the BPE algorithm finds many existing, learned subword tokens, resulting in a **short, compressed sequence** (e.g., 7 tokens in the notebook's example).
2.  **Low-Frequency Merges (English):** When an English sentence like **"why is the sky blue"** is passed to the same tokenizer:
    * Some common English subwords are **not** present in the learned Spanish dictionary.
    * The BPE algorithm falls back to the smaller, less efficient learned tokens, often resulting in sequences that are closer to **character-level tokenization**.
    * This yields a **longer, less compressed sequence** (more tokens) for the same number of conceptual "words," increasing the sequence length ($T$) and the associated $O(T^2)$ computational cost of the self-attention mechanism.

In [450]:
input_str = None

if not input_str:
    input_str = "por que es el cielo azul?"

def visualize_tokenizer(input_str):
    encoded_str = bp_encode(input_str,  merge_dict_rev)
    color_list = generate_diverse_true_color_ansi_codes(NUM_COLORS)
    color_map = dict(zip(set(encoded_str), color_list))
    
    t = {}
    for k, v in color_map.items():
        token = List([k])
        word = bp_decode(token, merge_dict)
        t[k] = word
    
    res = ""
    for token in encoded_str:
        token = token
        word = t[token].replace(" ","•")
        res += f"{color_map[token]}{word}{RESET}"
    
    print(res)
    print(encoded_str)

visualize_tokenizer("por que es el cielo azul?")
visualize_tokenizer("why is the sky blue?")

#r = input("Tokenizer visualisation:\ntry:\"por que es el cielo azul?\"\nthen: \"try a sentence in english!\"\n")#"Por que es el cielo azul?"

por•que•es•el•cielo•azul?
[337, 267, 1485, 1783, 417, 750, 63]
why•is•the•sky•blue?
[10228, 272, 592, 1313, 115, 107, 272, 98, 489, 101, 63]


## Rotary Positional Embeddings (RoPE)

The `grm` function is an implementation of the core mathematical component of **Rotary Positional Embeddings (RoPE)**, a technique introduced by the LLaMA family of models to efficiently integrate positional information into the self-attention mechanism, replacing the traditional absolute positional embeddings.

Unlike traditional positional encodings, RoPE applies a **rotation** to the Query ($\boldsymbol{Q}$) and Key ($\boldsymbol{K}$) vectors based on the token's absolute position in the sequence ($m$).

### Calculating the Rotation Frequencies

The core of RoPE is the rotation angle ($\theta_i$) for each dimension $i$ of the embedding. This is done by calculating a base frequency vector based on the odd/even indices of the embedding dimension.

1.  **Base Frequency Term:** The term $\frac{1}{\theta^{\frac{i}{d}}}$ is computed, where:
    * $i$ is the index of the embedding dimension, which is iterated over as $0, 2, 4, \dots, d-2$.
    * $d$ is the full embedding dimension $C$ (`n_embd`).
    * $\theta$ is a constant base (e.g., $10000$ or $\pi$ as shown in the tests).
2.  **Code Implementation (`grm` function):**
    * `i_indices = torch.arange(0, n_embd, 2)`: Generates the $i$ indices ($0, 2, \dots$).
    * `frequencies_to_rotate = 1.0 / (theta ** (i_indices / n_embd))`: Computes the base frequency term $\frac{1}{\theta^{\frac{i}{d}}}$.
    * `m = torch.arange(len_sequence)`: Generates the token position indices $m$ ($0, 1, 2, \dots, T-1$).
    * `all_frequencies = torch.outer(m, frequencies_to_rotate)`: Calculates the final rotation angle for every dimension $i$ and every position $m$:
        $$\phi_{m, i} = m \cdot \left(\frac{1}{\theta^{\frac{i}{d}}}\right)$$
3.  **Rotation Matrix Components:**
    * The `cos_rot` and `sin_rot` matrices are then computed as $\cos(\phi_{m, i})$ and $\sin(\phi_{m, i})$.
    * The shape of both `cos_rot` and `sin_rot` is $(\boldsymbol{T}, \boldsymbol{C}/\mathbf{2})$, where $T$ is `len_sequence` and $C/2$ is the number of paired rotation indices.

### Avoiding the Full Rotation Matrix

A naive, explicit implementation of RoPE would calculate the full 2D rotation matrix $R_m$ of size $(C, C)$ for every position $m$ and then multiply the input vector by it, $Q'_m = R_m Q_m$.

$$R_m = \begin{pmatrix} \cos(\phi_{m, 0}) & -\sin(\phi_{m, 0}) & 0 & 0 & \dots \\ \sin(\phi_{m, 0}) & \cos(\phi_{m, 0}) & 0 & 0 & \dots \\ 0 & 0 & \cos(\phi_{m, 2}) & -\sin(\phi_{m, 2}) & \dots \\ 0 & 0 & \sin(\phi_{m, 2}) & \cos(\phi_{m, 2}) & \dots \\ \dots & \dots & \dots & \dots & \dots \end{pmatrix}$$

The **optimized implementation** (as shown in the notebook's test cells) avoids this large matrix multiplication by exploiting the block-diagonal structure of $R_m$ and performing the rotation in place:

1.  **Split Vectors:** The input vector $x$ (Query or Key) is split into two halves: the **even indices** $x_{\text{even}}$ (dimensions $0, 2, \dots$) and the **odd indices** $x_{\text{odd}}$ (dimensions $1, 3, \dots$).
2.  **Apply Rotation Formula:** The rotation is applied directly using the pre-calculated `cos_rot` and `sin_rot` (which are of shape $(T, C/2)$):
    $$\text{Rotated}(x_0) = x_0 \cos(\phi) - x_1 \sin(\phi)$$
    $$\text{Rotated}(x_1) = x_1 \cos(\phi) + x_0 \sin(\phi)$$
3.  **Combine Rotated Halves:** The newly rotated even and odd vectors are interleaved back into the final vector $\boldsymbol{x}'$.

This approach ensures the positional encoding is applied with a **minimal number of multiplications and without the need to instantiate large rotation matrices**, significantly improving efficiency.

In [438]:
def grm(len_sequence:int, n_embd:int, theta:float=10000):
    i_indices = torch.arange(0, n_embd, 2)
    m = torch.arange(len_sequence)
    frequencies_to_rotate = 1.0 / (theta ** (i_indices / n_embd))

    all_frequencies = torch.outer(m, frequencies_to_rotate)
    cos_rot = torch.cos(all_frequencies)
    sin_rot = torch.sin(all_frequencies)
    return cos_rot, sin_rot

In [439]:
import math
n_embd = 8
cos_rot, sin_rot = grm(2, n_embd, math.pi)

In [440]:
x = torch.tensor([[0,1,2,3,4,5,6,7], [0,1,2,3,4,5,6,7]])
x_even = x[..., 0::2]
x_odd  = x[..., 1::2]
x_even_rot = (cos_rot * x_even - sin_rot * x_odd).bfloat16()
x_odd_rot = (cos_rot * x_odd + sin_rot * x_even).bfloat16()

In [441]:
final_rotated_x_slice = torch.empty((2, n_embd), dtype=x_even_rot.dtype) 
# 2. Assign the rotated even indices to the even columns (0, 2, 4, 6)
final_rotated_x_slice[:, 0::2] = x_even_rot 
# 3. Assign the rotated odd indices to the odd columns (1, 3, 5, 7)
final_rotated_x_slice[:, 1::2] = x_odd_rot

In [442]:
final_rotated_x_slice

tensor([[ 0.0000,  1.0000,  2.0000,  3.0000,  4.0000,  5.0000,  6.0000,  7.0000],
        [-0.8398,  0.5391, -0.5859,  3.5625,  0.7070,  6.3750,  2.5938,  8.8750]],
       dtype=torch.bfloat16)

In [197]:
num_layers = 4
n_embd = 1024 >> 2
num_heads = 12
block_size = 2048 >> 2
vocab_size = max(merge_dict)
gpt = NanoGPT(num_layers, vocab_size, n_embd, num_heads, block_size)
gpt.to(device)
print(sum(p.numel() for p in gpt.parameters())/1e6, 'M parameters')

8.531983 M parameters
